In [ ]:
import src.graph

In [ ]:
input = {
    'file_name': 'smaller_data.json',
    'user_id' : 'p007',
    'user_session_id' : '1',
    'data_info_from_user' : "json array of events triggered by all minecraft players in a server in choronological order",
    'messages' : []
}

In [ ]:
workflow_complied = src.graph.get_feild_name_correcting_task_graph()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(workflow_complied.get_graph().draw_mermaid_png()))
    #save it to a file
    workflow_complied.get_graph().draw_mermaid_png(output_file_path="workflow.png")
except Exception:
    pass

In [ ]:
out = workflow_complied.invoke(input)

In [ ]:
"""with open("temp/p007/1/out.json", "r") as f:
    data = f.read()
    print(len(data))"""

In [ ]:
"""with open("temp/p007/1/data.json", "r") as f:
    data = f.read()
    print(len(data))"""

In [ ]:
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import os, dotenv

dotenv.load_dotenv()

credentials = ServiceAccountCredentials.from_json_keyfile_name('gcreds.json')
client = storage.Client(credentials=credentials, project='assetgeneration')
bucket = client.get_bucket('xg_live_ops')

In [ ]:
bucket.blob(f'{input["user_id"]}/{input["user_session_id"]}/smaller_data.json').upload_from_filename("temp/p007/1/smaller_data.json")

In [ ]:
import json

In [ ]:
bucket.blob(f'{input["user_id"]}/{input["user_session_id"]}/out.json').upload_from_string(data=json.dumps(dict(out)), content_type="application/json")

In [ ]:
d = json.loads(out['field_info_list'][0].model_dump_json())

In [ ]:
import src.flat.flat.main

In [ ]:
src.flat.flat.main.main("temp/p007/1/", "original_copy.json")

In [4]:
import json 

In [5]:
def flatten_json_leaving_lists(data, parent_key='', sep='.'):
    """
    Flatten JSON data into a list of flat dictionaries, leaving list values as is.

    Parameters:
    data (dict, list, or any): The JSON data to flatten.
    parent_key (str): The base key string used during recursion.
    sep (str): Separator used when concatenating keys.

    Returns:
    list: A list of flat dictionaries.
    """
    flat_list = []

    if isinstance(data, dict):
        flat_dict = {}
        for k, v in data.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                # Flatten nested dictionary
                nested_flat_list = flatten_json_leaving_lists(v, new_key, sep=sep)
                if nested_flat_list:
                    flat_dict.update(nested_flat_list[0])
            elif isinstance(v, list):
                # Leave list values as is
                flat_dict[new_key] = v
            else:
                # Primitive value
                flat_dict[new_key] = v
        flat_list.append(flat_dict)
    elif isinstance(data, list):
        # Process each item in the list
        for item in data:
            item_flat_list = flatten_json_leaving_lists(item, parent_key=parent_key, sep=sep)
            flat_list.extend(item_flat_list)
    else:
        # Data is a primitive value
        flat_list.append({parent_key or 'value': data})

    return flat_list


In [6]:
with open("src/flat/test_jsons/t1.json", "r") as f:
    json_data = json.load(f)
    print(flatten_json_leaving_lists(json_data))

[{'event_001.event_type': 'player_join', 'event_001.player_id': 'player_12345', 'event_001.timestamp': '2024-11-21T10:00:00Z', 'event_001.location.x': 100, 'event_001.location.y': 64, 'event_001.location.z': -300, 'event_001.details.message': 'Player joined the game', 'event_002.event_type': 'block_break', 'event_002.player_id': 'player_12345', 'event_002.timestamp': '2024-11-21T10:05:00Z', 'event_002.location.x': 105, 'event_002.location.y': 65, 'event_002.location.z': -310, 'event_002.details.block_type': 'stone', 'event_002.details.tool_used': 'diamond_pickaxe', 'event_002.details.dropped_items': ['cobblestone'], 'event_003.event_type': 'chat_message', 'event_003.player_id': 'player_67890', 'event_003.timestamp': '2024-11-21T10:10:00Z', 'event_003.details.message': 'Anyone up for a raid?', 'event_003.details.chat_channel': 'global', 'event_004.event_type': 'player_death', 'event_004.player_id': 'player_12345', 'event_004.timestamp': '2024-11-21T10:15:00Z', 'event_004.location.x': 12

In [8]:
with open("src/flat/test_jsons/t2.json", "w") as f:
    json.dump(flatten_json_leaving_lists(json_data), f, indent=4)